In [1]:
# Import the required libraries and open the connection to Mongo

import collections
import datetime
import matplotlib as mpl

import pandas as pd
import scipy.stats

import folium

import pymongo

In [2]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels
roads = db.roads

In [3]:
# Load the expanded names of keys and human-readable codes into memory
expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

In [4]:
def add_accidents_markers(the_map, query, number_of_sides=5, fill_color='#769d96', limit=0,
                     radius=5, rotation=54):
    for a in accidents.find(query, 
                            ['loc.coordinates'],
                            limit=limit):
        folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=number_of_sides, radius=radius, rotation=rotation,
                                   fill_color=fill_color).add_to(the_map)  

In [5]:
def add_roads_markers(the_map, query, number_of_sides=5, fill_color='#769d96', limit=0,
                     radius=5, rotation=54):
    for r in roads.find(query, 
                        ['loc.coordinates'],
                       limit=limit):
        folium.RegularPolygonMarker(location=[r['loc']['coordinates'][1], r['loc']['coordinates'][0]], 
                     number_of_sides=number_of_sides, radius=radius, rotation=rotation,
                                   fill_color=fill_color).add_to(the_map)   

# Activity 1

In [6]:
pipeline = [{'$group': {'_id': '$RCat',
                        'FdAll_MV': {'$sum': '$FdAll_MV'}, 
                        'LenNet': {'$sum': '$LenNet'}, 
                        'nearby_accident_count': {'$sum': '$nearby_accident_count'}}},
            {'$project': {'accidents_per_km': {'$divide': ['$nearby_accident_count', '$LenNet']},
                          'accidents_per_vehicle_km': {'$divide': 
                                                       ['$nearby_accident_count', 
                                                        {'$multiply': ['$LenNet', '$FdAll_MV']}]}}}]

results = list(roads.aggregate(pipeline))
results_df = pd.DataFrame(results)
results_df.set_index('_id', inplace=True)
results_df

,accidents_per_km,accidents_per_vehicle_km
_id,,
PM,0,0
TU,0,0
PU,0,0
PR,0,0
TM,0,0
TR,0,0


# Activity 2

In [9]:
import math

# Plot all the road segments within 50km of Milton Keynes

m = folium.Map([52.0395099, -0.7601851], zoom_start=9)

query = {'loc': 
          {'$nearSphere':
           {'$geometry': 
            {'type': 'Point', 
             'coordinates': [-0.7601851, 52.0395099]},
            '$maxDistance': 50000}},
         'LenNet': {'$exists': True},
         'nearby_accident_count': {'$gt': 0}}

for r in roads.find(query):
    folium.RegularPolygonMarker(location=[r['loc']['coordinates'][1], r['loc']['coordinates'][0]], 
                     number_of_sides=9, fill_color='#ff0000',
                    radius=(math.log10(r['nearby_accident_count'] / (r['LenNet'] * r['FdAll_MV'])) + 6) * 3).add_to(m)  
m

# Activity 3

This question is deliberately vague. What is meant by "most dangerous"? It could be the segment with the most accidents, the most accidents per km, or the most accidents per vehicle-km.

In [ ]:
# Most accidents
pipeline = [{'$geoNear': {'near': {'type': 'Point', 'coordinates': [-0.7601851, 52.0395099]},
                          'spherical': True,
                          'maxDistance': 10000,
                          'distanceField': 'distance'}},
           {'$project': {'CP': '$CP', 
                         'Road': '$Road',
                         'nearby_accident_count': '$nearby_accident_count'}},
           {'$sort': {'nearby_accident_count': -1}},
           {'$limit': 1}]
results = list(roads.aggregate(pipeline))
results

In [ ]:
# Most accidents per km
pipeline = [{'$geoNear': {'near': {'type': 'Point', 'coordinates': [-0.7601851, 52.0395099]},
                          'spherical': True,
                          'maxDistance': 10000,
                          'distanceField': 'distance'}},
           {'$project': {'CP': '$CP', 
                         'Road': '$Road',
                         'accidents_per_km': {'$divide': ['$nearby_accident_count', '$LenNet']}}},
           {'$sort': {'accidents_per_km': -1}},
           {'$limit': 1}]
results = list(roads.aggregate(pipeline))
results

In [ ]:
# Most accidents per vehicle-km
pipeline = [{'$geoNear': {'near': {'type': 'Point', 'coordinates': [-0.7601851, 52.0395099]},
                          'spherical': True,
                          'maxDistance': 10000,
                          'distanceField': 'distance'}},
           {'$project': {'CP': '$CP', 
                         'Road': '$Road',
                         'accidents_per_vehicle_km': 
                             {'$divide': ['$nearby_accident_count', 
                                          {'$multiply': ['$LenNet', '$FdAll_MV']}]}}},
           {'$sort': {'accidents_per_vehicle_km': -1}},
           {'$limit': 1}]
results = list(roads.aggregate(pipeline))
results

# Activity 4

We need to count the number of casualties for each casualty class and road category. We can get that as a DataFrame by using a `crosstab` on a list of <casualty-class> <road-category> pairs.

As we can go from road to accidents using the `nearby-accidents` list, we should start by going through each road section in order, finding all the accidents near it, then finding all the casualities in each accident, and putting the results in some data struture. 

The natural approach is to use Mongo's aggregation pipeline, but that acts on only one collection at a time. Instead, we need to manually unwind the query from `road` to `accident` to each casualty sub-document in the accident.

We could do this verbosely with some nested `for` loops:

In [ ]:
casualty_class_by_road_cat_list = []

for r in roads.find({'nearby_accidents': {'$exists': True}}):
    for ai in r['nearby_accidents']:
        a = accidents.find_one({'Accident_Index': ai})
        for c in a['Casualties']:
            casualty_class_by_road_cat_list += [{'RCat': r['RCat'], 
                'Casualty_Class': label_of[('Casualty_Class', c['Casualty_Class'])]}]

casualty_class_by_road_cat_unrolled_df = pd.DataFrame(casualty_class_by_road_cat_list)
casualty_class_by_road_cat_unrolled_df

A more compact way of doing it is with a list comprehension, using the "trick" that we can use `accidents.find()` instead of `accidents.find_one()` to avoid the complication of trying to assign a temporary variable in the middle of the processing. 

In [ ]:
casualty_class_by_road_cat_unrolled_df = pd.DataFrame([
 {'RCat': r['RCat'], 'Casualty_Class': label_of[('Casualty_Class', c['Casualty_Class'])]}
 for r in roads.find({'nearby_accidents': {'$exists': True}})
 for ai in r['nearby_accidents'] 
 for a in accidents.find({'Accident_Index': ai})
 for c in a['Casualties']
 ])

Either way, we then form the crosstab and move on to the rest of the processing.

In [ ]:
# Count the number of each severity
casualty_class_by_road_cat_df = pd.crosstab(casualty_class_by_road_cat_unrolled_df['RCat'], 
                                      casualty_class_by_road_cat_unrolled_df['Casualty_Class'])
casualty_class_by_road_cat_df

In [ ]:
casualty_class_by_road_cat_df.plot(kind='bar')

If we want to find statistical significance, we'll need to do a test. As the data is indexed by categories, a chi-squared test is appropriate. First, we calculate the expected number of casualties for each class and each road type, then perform the test by comparing the two DataFrames.

In [ ]:
# From notebook 14.3
def expected_of_df(actual_df):
    df = pd.DataFrame(
        {c: 
         {r: actual_df[c].sum() * actual_df.loc[r].sum() / actual_df.sum().sum()
                  for r in actual_df[c].index} 
              for c in actual_df})
    # Fix the order of columns and rows
    df = df[actual_df.columns]
    df = df.reindex(actual_df.index)
    return df

In [ ]:
expected_casualty_class_by_road_cat_df = expected_of_df(casualty_class_by_road_cat_df)
expected_casualty_class_by_road_cat_df

In [ ]:
scipy.stats.chisquare(casualty_class_by_road_cat_df, expected_casualty_class_by_road_cat_df, axis=None)

Yes, different types of road do have different mixes of casualties.